# Covid-19: From model prediction to model predictive control

## Calibration of the age-stratified national model (google)

*Original code by Ryan S. McGee. Modified by T.W. Alleman in consultation with the BIOMATH research unit headed by prof. Ingmar Nopens.*

Copyright (c) 2020 by T.W. Alleman, BIOMATH, Ghent University. All Rights Reserved.

This notebook accompanies our preprint: "*A deterministic, age-stratified, extended SEIRD model for assessing the effect of non-pharmaceutical interventions on SARS-CoV-2 spread in Belgium*"(https://doi.org/10.1101/2020.07.17.20156034)

### Load required packages

In [1]:
import random
import os
import numpy as np
import json
import corner
import random

import pandas as pd
import datetime
import scipy
import matplotlib.dates as mdates
import matplotlib
import math
import xarray as xr
import emcee
import matplotlib.pyplot as plt
import datetime

from covid19model.optimization import objective_fcns,pso
from covid19model.models import models
from covid19model.models.utils import draw_sample_COVID19_SEIRD_google
from covid19model.models.time_dependant_parameter_fncs import google_lockdown, google_lockdown_no_prev
from covid19model.data import google
from covid19model.data import sciensano
from covid19model.data import model_parameters
from covid19model.visualization.output import population_status, infected, _apply_tick_locator 
from covid19model.visualization.optimization import plot_fit, traceplot

# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

# Load data

### Load interaction matrices

In [2]:
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(dataset='willem_2012')
levels = initN.size

### Scrape high-level Sciensano data

In [3]:
df_sciensano = sciensano.get_sciensano_COVID19_data(update=False)
df_sciensano.tail(2)

,H_tot,ICU_tot,H_in,H_out,H_tot_cumsum,D_tot,D_25_44,D_45_64,D_65_74,D_75_84,D_85+
DATE,,,,,,,,,,,
2020-11-24,4820,1105,286,528,4594,83,0.0,12.0,10,24,37
2020-11-25,4570,1071,272,429,4437,17,1.0,4.0,3,3,6


### Scrape Google Community Mobility Reports

In [4]:
df_google = google.get_google_mobility_data(update=False, plot=False)
df_google.tail(2)

,retail_recreation,grocery,parks,transport,work,residential
date,,,,,,
2020-11-21,-58.0,-11.0,13.0,-41.0,-20.0,13.0
2020-11-22,-63.0,-18.0,-11.0,-45.0,-21.0,10.0


### Load time-dependant parameters

In [5]:
from covid19model.models.time_dependant_parameter_fncs import google_lockdown

def switch_beta(t,param,samples_dict):
    if t < pd.to_datetime('2020-05-04'):
        return np.random.choice(samples_dict['beta'],1,replace=False)
    elif pd.to_datetime('2020-05-04') < t <= pd.to_datetime('2020-09-01'):
        return np.random.choice(samples_dict['beta_summer'],1,replace=False)
    else:
        return np.random.choice(samples_dict['beta'],1,replace=False)

# Wave 1 - March 2020 - September 2020

### Initialize the model using the google lockdown function

In [7]:
# Load samples dictionary of the first wave
with open('../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/classical/BE_2020-11-05.json', 'r') as fp:
    samples_dict = json.load(fp)

In [8]:
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(dataset='willem_2012',intensity='all')
initN, Nc_home_15, Nc_work_15, Nc_schools_15, Nc_transport_15, Nc_leisure_15, Nc_others_15, Nc_total_15 = model_parameters.get_interaction_matrices(dataset='willem_2012',intensity='more_15_min')
initN, Nc_home_1hr, Nc_work_1hr, Nc_schools_1hr, Nc_transport_1hr, Nc_leisure_1hr, Nc_others_1hr, Nc_total_1hr = model_parameters.get_interaction_matrices(dataset='willem_2012',intensity='more_one_hour')

In [9]:
Nc_all = {'total': Nc_total, 'home':Nc_home, 'work': Nc_work, 'schools': Nc_schools, 'transport': Nc_transport, 'leisure': Nc_leisure, 'others': Nc_others}
Nc_15min = {'total': Nc_total_15, 'home': Nc_home_15, 'work': Nc_work_15, 'schools': Nc_schools_15, 'transport': Nc_transport_15, 'leisure': Nc_leisure_15, 'others': Nc_others_15}
Nc_1hr = {'total': Nc_total_1hr, 'home': Nc_home_1hr, 'work': Nc_work_1hr, 'schools': Nc_schools_1hr, 'transport': Nc_transport_1hr, 'leisure': Nc_leisure_1hr, 'others': Nc_others_1hr}

In [11]:
# Load the model parameters using `get_COVID19_SEIRD_parameters()`.
params = model_parameters.get_COVID19_SEIRD_parameters()

params.update({'df_google': df_google,
              'Nc_all' : Nc_all,
              'Nc_15min' : Nc_15min,
              'Nc_1hr' : Nc_1hr,
               'l' : 5,
               'tau' : 5,
              })

# Define the initial condition: one exposed inidividual in every age category
initial_states = {'S': initN, 'E': np.ones(levels)}
# Initialize the model
model = models.COVID19_SEIRD(initial_states, params, time_dependent_parameters={'Nc': google_lockdown_no_prev})

## Perform calibration

In [12]:
# Date of first data collection
start_data = '2020-03-15'
# Last datapoint used to calibrate basic reproduction number
end_calibration = '2020-05-04'
# Start of recalibration beta
start_recalibrate_beta = end_calibration
# End of recalibration beta
end_recalibrate_beta = '2020-09-01'

# Path where figures should be stored
fig_path = '../../results/calibrations/COVID19_SEIRD/national/'
# Path where MCMC samples should be saved
samples_path = '../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/'
# Spatial unit: Belgium
spatial_unit = 'BE_no_prev'

In [ ]:
start_data

In [22]:
from covid19model.optimization.run_optimization import google_calibration_wave1
samples_dict = google_calibration_wave1(model, df_sciensano['H_in'], spatial_unit, start_data, end_calibration, start_recalibrate_beta,
                        end_recalibrate_beta, fig_path, 
                        samples_path, initN, Nc_total,warmup=samples_dict['warmup'],maxiter=100, popsize=100, steps_mcmc=5000,
                        discard=1000, omega=0.8, phip=0.8, phig=0.8, processes=32)

------------------------------------
CALIBRATING BETA AND COMPLIANCE RAMP
------------------------------------

Using data from 2020-03-15 until 2020-05-04

1) Particle swarm optimization

No constraints given.
Best after iteration 1: [7.82710369e+01 3.95086576e-02 2.16930304e+00 9.70991078e+00] 239.27004777699824
Best after iteration 2: [7.82710369e+01 3.95086576e-02 2.16930304e+00 9.70991078e+00] 239.27004777699824
New best for swarm at iteration 3: [7.99819011e+01 3.99606185e-02 1.00000000e-02 1.08104879e+01] 235.83841007979845
Best after iteration 3: [7.99819011e+01 3.99606185e-02 1.00000000e-02 1.08104879e+01] 235.83841007979845
New best for swarm at iteration 4: [5.42384704e+01 4.00000000e-02 1.00000000e-02 1.06652661e+01] 231.24265488301805
Best after iteration 4: [5.42384704e+01 4.00000000e-02 1.00000000e-02 1.06652661e+01] 231.24265488301805
Best after iteration 5: [5.42384704e+01 4.00000000e-02 1.00000000e-02 1.06652661e+01] 231.24265488301805
Best after iteration 6: [5.42384

Best after iteration 62: [5.53392630e+01 3.98483437e-02 1.00000000e-02 1.10545139e+01] 229.8117522087092
Best after iteration 63: [5.53392630e+01 3.98483437e-02 1.00000000e-02 1.10545139e+01] 229.8117522087092
Best after iteration 64: [5.53392630e+01 3.98483437e-02 1.00000000e-02 1.10545139e+01] 229.8117522087092
Best after iteration 65: [5.53392630e+01 3.98483437e-02 1.00000000e-02 1.10545139e+01] 229.8117522087092
Best after iteration 66: [5.53392630e+01 3.98483437e-02 1.00000000e-02 1.10545139e+01] 229.8117522087092
Best after iteration 67: [5.53392630e+01 3.98483437e-02 1.00000000e-02 1.10545139e+01] 229.8117522087092
Best after iteration 68: [5.53392630e+01 3.98483437e-02 1.00000000e-02 1.10545139e+01] 229.8117522087092
Best after iteration 69: [5.53392630e+01 3.98483437e-02 1.00000000e-02 1.10545139e+01] 229.8117522087092
Best after iteration 70: [5.53392630e+01 3.98483437e-02 1.00000000e-02 1.10545139e+01] 229.8117522087092
Best after iteration 71: [5.53392630e+01 3.98483437e-02

  0%|          | 0/5000 [00:00<?, ?it/s]/home/data/jvergeyn/.conda/envs/COVID_MODEL/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]
100%|██████████| 5000/5000 [5:33:56<00:00,  4.01s/it]  

Use this estimate with caution and run a longer chain!


ValueError: slice step cannot be zero

## Visualizations

### Goodness-of-fit: $\beta$, $l$ and $\tau$

In [ ]:
params.update({'samples_dict': samples_dict})

# Re-initialize the model
model = models.COVID19_SEIRD(initial_states, params, time_dependent_parameters={'Nc': google_lockdown, 'beta': switch_beta})

In [ ]:
end_sim = '2020-09-01'

fig,ax=plt.subplots(figsize=(10,4))
for i in range(10):
    model.parameters = draw_sample_COVID19_SEIRD_google(model.parameters,samples_dict)
    y_model = model.sim(end_sim,start_date=start_data,excess_time=samples_dict['warmup'])
    ax.plot(y_model['time'],y_model["H_in"].sum(dim="Nc"),color='blue',alpha=0.01)

ax.scatter(df_sciensano[:end_calibration].index,df_sciensano['H_in'][:end_calibration],color='black',alpha=0.6,linestyle='None',facecolors='none')
ax.scatter(df_sciensano[end_calibration:end_sim].index,df_sciensano['H_in'][end_calibration:end_sim],color='red',alpha=0.6,linestyle='None',facecolors='none')
ax = _apply_tick_locator(ax)
ax.set_xlim('2020-03-01',end_sim)

plt.savefig(fig_path+'others/BETA_RAMP_FIT_WAVE1_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')

### $R_0$

In [ ]:
fig,ax=plt.subplots(figsize=(10,4))
ax.hist(samples_dict['R0'],bins=30)
plt.savefig(fig_path+'others/R0_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')

### Seroprelevance

In [ ]:
end_sim = '2020-06-01'

fig,ax=plt.subplots(figsize=(10,4))
for i in range(10):
    model.parameters = draw_sample_COVID19_SEIRD_google(model.parameters,samples_dict)
    y_model = model.sim(end_sim,start_date=start_data,excess_time=samples_dict['warmup'])
    ax.plot((1-y_model["S"].sel(time=end_sim)/y_model["S"].isel(time=0))*100,alpha=0.03,color='blue',linewidth=2)

ax.set_xlabel('age')
ax.set_ylabel("seroprelevance (%)" )
ax.set_ylim()
plt.xticks(np.arange(9), ('[0-10[', '[10-20[', '[20-30[', '[30-40[', '[40-50[', '[50-60[', '[60-70[', '[70-80[', '[80-$\infty$['))

plt.savefig(fig_path+'others/SEROPRELEVANCE_'+end_sim+'_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')

### Lockdown release

In [ ]:
end_sim = '2020-09-01'
n=100
percentile = 0.99

y_model = model.sim(end_sim,start_date=start_data,excess_time=samples_dict['warmup'],N=n,draw_fcn=draw_sample_COVID19_SEIRD_google,samples=samples_dict)

fig,ax = plt.subplots(figsize=(10,4))
ax.fill_between(pd.to_datetime(y_model['time'].values),y_model["H_tot"].quantile(1-percentile,dim="draws").sum(dim="Nc"), y_model["H_tot"].quantile(percentile,dim="draws").sum(dim="Nc"),alpha=0.30)
ax.plot(y_model['time'],y_model["H_tot"].mean(dim="draws").sum(dim="Nc"),'--')
ax.scatter(df_sciensano[:end_sim].index,df_sciensano['H_tot'][:end_sim],color='black',alpha=0.4,linestyle='None',facecolors='none')
ax = _apply_tick_locator(ax)
plt.savefig(fig_path+'others/LOCKDOWN_RELEASE_WAVE1_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')

### Save results to a .csv

In [ ]:
index = y_model['time'].values
columns = ['WAVE1_INCIDENCES_MEAN','WAVE1_INCIDENCES_MEDIAN','WAVE1_INCIDENCES_LL','WAVE1_INCIDENCES_UL']
data = np.transpose(np.array([
        y_model["H_tot"].mean(dim="draws").sum(dim="Nc").values.tolist(),
        y_model["H_tot"].median(dim="draws").sum(dim="Nc").values.tolist(),
        y_model["H_tot"].quantile(1-percentile,dim="draws").sum(dim="Nc").values.tolist(),
        y_model["H_tot"].quantile(percentile,dim="draws").sum(dim="Nc").values.tolist()
       ]))

In [ ]:
df = pd.DataFrame(data,columns=columns,index=index)
df.head()

In [ ]:
df.to_csv('../../results/predictions/FAGG/COVID19_SEIRD/WAVE1.csv',index=True)

# Wave 2: September 2020 - present

## Recalibrate ramp as final calibration step

In [ ]:
# Load samples dictionary of the first wave
with open('../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/google/BE_2020-11-10_WAVE1_GOOGLE.json', 'r') as fp:
    samples_dict = json.load(fp)

with open('../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/google/initial_states_2020-09-01.json', 'r') as fp:
    initial_states = json.load(fp)    

In [ ]:
# Start of data collection
start_data = '2020-09-01'
# Start data of recalibration ramp
start_calibration = '2020-09-01'
# Last datapoint used to recalibrate the ramp
end_calibration = '2020-11-23'
# Path where figures should be stored
fig_path = '../../results/calibrations/COVID19_SEIRD/national/'
# Path where MCMC samples should be saved
samples_path = '../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/'
# Spatial unit: Belgium
spatial_unit = 'BE'

In [ ]:
# Load the model parameters using `get_COVID19_SEIRD_parameters()`.
params = model_parameters.get_COVID19_SEIRD_parameters()

params.update({'df_google': df_google,
              'Nc_all' : Nc_all,
              'Nc_15min' : Nc_15min,
              'Nc_1hr' : Nc_1hr,
               'l' : 5,
               'tau' : 5,
               'prevention': 0.8
              })

# Initialize the model
model = models.COVID19_SEIRD(initial_states, params, time_dependent_parameters={'Nc': google_lockdown})

In [ ]:
warmup=0
maxiter = 100
popsize = 100
steps_mcmc = 3000
discard = 1000

# define dataset
data=[df_sciensano['H_in'][start_calibration:end_calibration]]
states = [["H_in"]]

####################################################
####### CALIBRATING BETA AND COMPLIANCE RAMP #######
####################################################

print('------------------------------------')
print('CALIBRATING BETA AND COMPLIANCE RAMP')
print('------------------------------------\n')
print('Using data from '+start_calibration+' until '+end_calibration+'\n')
print('1) Particle swarm optimization\n')

# set PSO optimisation settings
parNames = ['sigma_data','beta','l','tau','prevention']
bounds=((1,2000),(0.010,0.060),(0.1,20),(0.1,20),(0,1))
# run PSO optimisation
theta = pso.fit_pso(model,data,parNames,states,bounds,maxiter=maxiter,popsize=popsize,start_date=start_calibration,warmup=warmup)


In [ ]:

# run MCMC sampler
print('\n2) Markov-Chain Monte-Carlo sampling\n')
parNames_mcmc = parNames
bounds_mcmc=((1,2000),(0.020,0.060),(0.001,20),(0.001,20),(0,1))

pos = theta + [1, 1e-3, 1e-3, 1e-3, 1e-3 ]* np.random.randn(10, 5)
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(nwalkers, ndim, objective_fcns.log_probability,
                    args=(model, bounds_mcmc, data, states, parNames_mcmc, None, start_calibration, warmup))
sampler.run_mcmc(pos, steps_mcmc, progress=True)

try:
    sampler.get_autocorr_time()
except:
    print('Warning: The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s).\nUse this estimate with caution and run a longer chain!')
from covid19model.optimization.run_optimization import checkplots
checkplots(sampler, discard, fig_path, spatial_unit, 
            figname='BETA_RAMP_GOOGLE_WAVE2', labels=['$\sigma_{data}$','$\\beta$','l','$\\tau$','$\Sigma$'])

#############################################
####### Output to dictionary ################
#############################################

samples_dict_wave2 = {
    'beta': sampler.get_chain(discard=discard,flat=True)[:,1].tolist(),
    'l' : sampler.get_chain(discard=discard,flat=True)[:,2].tolist(),
    'tau' : sampler.get_chain(discard=discard,flat=True)[:,3].tolist(),
    'prevention': sampler.get_chain(discard=discard,flat=True)[:,4].tolist(),
}

with open(samples_path+str(spatial_unit)+'_'+str(datetime.date.today())+'_WAVE2_GOOGLE.json', 'w') as fp:
    json.dump(samples_dict_wave2, fp)

## Visualize fit

In [ ]:
# Load samples dictionary of the first wave
with open('../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/BE_2020-11-26_WAVE2_GOOGLE.json', 'r') as fp:
    samples_dict_wave2 = json.load(fp)

In [ ]:
end_sim = '2021-01-01'

fig,ax=plt.subplots(figsize=(10,4))
for i in range(100):
    # Sampling
    idx, model.parameters['beta'] = random.choice(list(enumerate(samples_dict_wave2['beta'])))
    model.parameters['l'] = samples_dict_wave2['l'][idx] 
    model.parameters['tau'] = samples_dict_wave2['tau'][idx]  
    model.parameters['prevention'] = samples_dict_wave2['prevention'][idx]  
   
    # Simulate
    y_model = model.sim(end_sim,start_date=start_calibration,warmup=0)
    # Plot
    ax.plot(y_model['time'],y_model["H_in"].sum(dim="Nc"),color='blue',alpha=0.1)

ax.scatter(df_sciensano[start_calibration:end_calibration].index,df_sciensano['H_in'][start_calibration:end_calibration],color='black',alpha=0.6,linestyle='None',facecolors='none')
ax = _apply_tick_locator(ax)
ax.set_xlim('2020-09-01',end_sim)
plt.savefig(fig_path+'others/BETA_RAMP_FIT_WAVE2_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')

## Make a prediction under current contact behaviour

In [ ]:
end_sim = '2021-02-01'
n=200
percentile = 0.95

y_model = model.sim(end_sim,start_date=start_calibration,warmup=0,N=n,draw_fcn=draw_sample_COVID19_SEIRD_google,samples=samples_dict_wave2)

fig,ax = plt.subplots(figsize=(10,4))
ax.fill_between(pd.to_datetime(y_model['time'].values),y_model["H_tot"].quantile(1-percentile,dim="draws").sum(dim="Nc"), y_model["H_tot"].quantile(percentile,dim="draws").sum(dim="Nc"),alpha=0.20, color = 'blue')
ax.fill_between(pd.to_datetime(y_model['time'].values),y_model["H_tot"].quantile(1-0.68,dim="draws").sum(dim="Nc"), y_model["H_tot"].quantile(0.68,dim="draws").sum(dim="Nc"),alpha=0.35, color = 'blue')
ax.plot(y_model['time'],y_model["H_tot"].mean(dim="draws").sum(dim="Nc"),'--', color='blue')
ax.scatter(df_sciensano[start_calibration:end_calibration].index,df_sciensano['H_tot'][start_calibration:end_calibration],color='black',alpha=0.4,linestyle='None',facecolors='none')
ax.set_ylabel('Total patients in Belgian hospitals')
ax = _apply_tick_locator(ax)
plt.savefig(fig_path+'others/LOCKDOWN_WAVE2_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')